In [1]:
import os
import random
from glob import glob
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity as ssim
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import cv2
import copy
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Paths to dataset images and ground truths
dataset_path = r'CHASEDB1'
image_files = glob(os.path.join(dataset_path, '*.jpg'))
ground_truth_files = glob(os.path.join(dataset_path, '*_1stHO.png'))
image_files.sort()
ground_truth_files.sort()

In [3]:
def accuracy_func(image1, image2):
    segmented_vessels = image1
    segmented_image = image2

    ssim_score = ssim(segmented_image, segmented_vessels, data_range=255)
    mse = np.mean((image1 - image2) ** 2)
    psnr_score = 20 * np.log10(255.0 / np.sqrt(mse)) if mse != 0 else float('inf')

    generated_binary = (segmented_vessels > 0).astype(np.uint8)
    target_binary = (segmented_image > 0).astype(np.uint8)

    tn, fp, fn, tp = confusion_matrix(target_binary.ravel(), generated_binary.ravel()).ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1 = 2 / ((1 / specificity) + (1 / sensitivity))
    
    return accuracy, sensitivity, specificity, f1, ssim_score, psnr_score

In [4]:
def threshold(img,k):
    ret = copy.deepcopy(img)
    ret[ret<k] = 0
    ret[ret>=k] = 255
    return ret

def GlobalOtsu(img):
    foreground = img[img>=0]
    background = img[img<0]
    
    final_var = (np.var(foreground) * len(foreground) + np.var(background) * len(background))/(len(foreground) + len(background))
    if(np.isnan(final_var)):
        final_var = -1
        
    final_thresh = 0
    for i in np.linspace(np.min(img), np.max(img), num=255):
        foreground = img[img>=i]
        background = img[img<i]
        var = (np.var(foreground) * len(foreground) + np.var(background) * len(background))/(len(foreground) + len(background))
        
        if(np.isnan(var)):
            var = -1
            
        if(var!=-1 and (var<final_var or final_var ==-1)):
            final_var = var
            final_thresh = i
    return threshold(img,final_thresh)

In [5]:
metrics_grayscale = []
metrics_green = []

# selected_indices = random.sample(range(len(image_files)), 10)

for idx , img in enumerate(image_files):
    img_path = image_files[idx]
    ground_img_path = ground_truth_files[idx]

    img = Image.open(img_path)
    ground_img = Image.open(ground_img_path)
    img_array = np.array(img)
    ground_img_array = np.array(ground_img)

    img_gray_array = np.array(img.convert('L'))
    green_channel = img_array[:, :, 1]

    brightness_factor = random.uniform(0.8, 1.6)
    scaled_green_channel = np.clip(green_channel * brightness_factor, 0, 255).astype(np.uint8)

    clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(8, 8))
    cl_img_gray = clahe.apply(img_gray_array)
    cl_img_green = clahe.apply(scaled_green_channel)

    cell_disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    Topen_gray = cv2.morphologyEx(cl_img_gray, cv2.MORPH_OPEN, cell_disc)
    Tclose_gray = cv2.morphologyEx(Topen_gray, cv2.MORPH_CLOSE, cell_disc)
    TopHat_gray = cl_img_gray - Tclose_gray
    min_image_gray = cv2.dilate(cv2.erode(TopHat_gray, kernel), kernel)

    Topen_green = cv2.morphologyEx(cl_img_green, cv2.MORPH_OPEN, cell_disc)
    Tclose_green = cv2.morphologyEx(Topen_green, cv2.MORPH_CLOSE, cell_disc)
    TopHat_green = cl_img_green - Tclose_green
    min_image_green = cv2.dilate(cv2.erode(TopHat_green, kernel), kernel)

    HessWide_gray = hessian_matrix(min_image_gray, sigma=4, order='rc')
    EignWide_gray = hessian_matrix_eigvals(HessWide_gray)[1]
    HessWide_green = hessian_matrix(min_image_green, sigma=4, order='rc')
    EignWide_green = hessian_matrix_eigvals(HessWide_green)[1]

    val_gray = GlobalOtsu(1 - EignWide_gray)
    val_green = GlobalOtsu(1 - EignWide_green)

    metrics_grayscale.append(accuracy_func(ground_img_array, val_gray))
    metrics_green.append(accuracy_func(ground_img_array, val_green))


metrics_grayscale = np.mean(metrics_grayscale, axis=0)
metrics_green = np.mean(metrics_green, axis=0)

print("Average Metrics (Grayscale):")
print(f"Accuracy: {metrics_grayscale[0]:.4f}, Sensitivity: {metrics_grayscale[1]:.4f}, Specificity: {metrics_grayscale[2]:.4f}, F1-Score: {metrics_grayscale[3]:.4f}, SSIM: {metrics_grayscale[4]:.4f}, PSNR: {metrics_grayscale[5]:.4f}")

print("\nAverage Metrics (Green Channel):")
print(f"Accuracy: {metrics_green[0]:.4f}, Sensitivity: {metrics_green[1]:.4f}, Specificity: {metrics_green[2]:.4f}, F1-Score: {metrics_green[3]:.4f}, SSIM: {metrics_green[4]:.4f}, PSNR: {metrics_green[5]:.4f}")


Average Metrics (Grayscale):
Accuracy: 0.9132, Sensitivity: 0.4109, Specificity: 0.9656, F1-Score: 0.5741, SSIM: 0.7356, PSNR: 10.3154

Average Metrics (Green Channel):
Accuracy: 0.9165, Sensitivity: 0.4287, Specificity: 0.9666, F1-Score: 0.5906, SSIM: 0.7428, PSNR: 10.3910
